In [5]:
import importlib
import tiny_video_dit
import dit_training
import video_dataset
import torch
from torch.utils.data import DataLoader, random_split

importlib.reload(tiny_video_dit)
importlib.reload(dit_training)
importlib.reload(video_dataset)

from tiny_video_dit import VideoDiT
from dit_training import train_video_dit
from video_dataset import VideoDataset

In [6]:
device = torch.device("cpu")

In [7]:
dataset = VideoDataset("downsampled_psft_videos")
train_size = 800
val_size = 200
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])
trainloader = DataLoader(train_dataset, batch_size=4, shuffle=True, num_workers=4)
valloader = DataLoader(val_dataset, batch_size=4, shuffle=False, num_workers=4)
model = VideoDiT().to(device)

In [8]:
num_epochs = 10
lr = 1e-4

train_video_dit(model, trainloader, valloader, num_epochs=num_epochs, lr=lr, device=device)

Epoch 1/10:   0%|          | 0/200 [00:00<?, ?batch/s]

Shape of pred_noise: torch.Size([4, 16, 64, 64, 3])
Shape of noise: torch.Size([4, 16, 64, 64, 3])


Epoch 1/10:   0%|          | 1/200 [00:02<09:00,  2.72s/batch, loss=1.31]

Shape of pred_noise: torch.Size([4, 16, 64, 64, 3])
Shape of noise: torch.Size([4, 16, 64, 64, 3])


Epoch 1/10:   1%|          | 2/200 [00:04<06:16,  1.90s/batch, loss=1.19]

Shape of pred_noise: torch.Size([4, 16, 64, 64, 3])
Shape of noise: torch.Size([4, 16, 64, 64, 3])


Epoch 1/10:   2%|▏         | 3/200 [00:05<05:16,  1.60s/batch, loss=1.13]

Shape of pred_noise: torch.Size([4, 16, 64, 64, 3])
Shape of noise: torch.Size([4, 16, 64, 64, 3])
